# ETL desde Kaggle para `countries-intermediate-dataset`

In [39]:
# Importamos librerías
import pandas as pd  # import pandas
import matplotlib.pyplot as plt  # import matplotlib
import seaborn as sns  # import seaborn
import utils as ut  # import utils
import kaggle  # import kaggle
import zipfile

In [40]:
# !kaggle datasets download -d <nombre-de-usuario>/<nombre-del-dataset>
!kaggle datasets download -d martj42/international-football-results-from-1872-to-2017

international-football-results-from-1872-to-2017.zip: Skipping, found more recently modified local copy (use --force to force download)


In [41]:
with zipfile.ZipFile(
    "international-football-results-from-1872-to-2017.zip", "r"
) as zip_ref:
    zip_ref.extractall(".")

## Como se tienem tres datasets, para este taller en particular nos enfocaremos en el dataset `goalscorers`

In [42]:
df = pd.read_csv("goalscorers.csv")

In [43]:
print(df.shape)
df.tail()

(44110, 8)


,date,home_team,away_team,team,scorer,minute,own_goal,penalty
44105,2024-02-10,Qatar,Jordan,Qatar,Akram Afif,73.0,False,True
44106,2024-02-10,Qatar,Jordan,Qatar,Akram Afif,90.0,False,True
44107,2024-02-11,Ivory Coast,Nigeria,Nigeria,William Troost-Ekong,38.0,False,False
44108,2024-02-11,Ivory Coast,Nigeria,Ivory Coast,Franck Kessié,62.0,False,False
44109,2024-02-11,Ivory Coast,Nigeria,Ivory Coast,Sébastien Haller,81.0,False,False


In [44]:
ut.verificar_tipo_datos_y_nulos(df)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
5,minute,[<class 'float'>],99.40,0.60,263
4,scorer,"[<class 'str'>, <class 'float'>]",99.89,0.11,50
0,date,[<class 'str'>],100.00,0.00,0
1,home_team,[<class 'str'>],100.00,0.00,0
2,away_team,[<class 'str'>],100.00,0.00,0
3,team,[<class 'str'>],100.00,0.00,0
6,own_goal,"[<class 'bool'>, <class 'float'>]",100.00,0.00,2
7,penalty,"[<class 'bool'>, <class 'float'>]",100.00,0.00,2


## Diccionario de datos

|   Columna   |          Descripción             |
|-------------|---------------------------------|
|     date    |  Fecha del partido              |
|  home_team  |  Nombre del equipo local        |
|  away_team  |  Nombre del equipo visitante    |
|    team     |  Nombre del equipo que anotó    |
|   scorer    |  Nombre del jugador que anotó   |
|  own_goal   |  Si el gol fue en propia meta   |
|   penalty   |  Si el gol fue de penal         |


## Contexto del dataset

Este dataset contiene información de los goleadores del futbol internacional (de selecciones), donde muestra el equipo donde jugó, si fue penalty, autogol y el minuto en que anotó gol.

## Objetivo

Se necesita saber cuál es el mayor goleador de encuentros de selecciones  desde 1872 hasta 2024, esto para clasificación y realización de un top de goleadores en sus respectivas selecciones nacionales. También un análisis de los equipos con más goleadores en sus plantillas

### Limpieza de nulos

Se eliminan los valores nulos pero donde no se encuentra el nombre del goleador, se dejan los valores nulos de minute, que es el minuto donde anotaron, ya que aunque se desconozca, se sabe que el jugador anotó y nos sirve para el objetivo final.

In [45]:
df.dropna(
    subset=["scorer"], inplace=True
)  # Eliminamos los nulos con base en la columna 'scorer'

In [46]:
ut.verificar_tipo_datos_y_nulos(df)  # Verificamos que se hayan eliminado los nulos

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
5,minute,[<class 'float'>],99.52,0.48,213
0,date,[<class 'str'>],100.00,0.00,0
1,home_team,[<class 'str'>],100.00,0.00,0
2,away_team,[<class 'str'>],100.00,0.00,0
3,team,[<class 'str'>],100.00,0.00,0
4,scorer,[<class 'str'>],100.00,0.00,0
6,own_goal,"[<class 'bool'>, <class 'float'>]",100.00,0.00,1
7,penalty,"[<class 'bool'>, <class 'float'>]",100.00,0.00,1


Dado que no es ni el 1 % de los datos los valores nulos de minute, decidimos imputar el valor promedio del dataset a estos registros vacíos

In [47]:
df["minute"].fillna(df["minute"].mean(), inplace=True)

In [48]:
ut.verificar_tipo_datos_y_nulos(df)  # Verificamos que se hayan eliminado los nulos

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,date,[<class 'str'>],100.0,0.0,0
1,home_team,[<class 'str'>],100.0,0.0,0
2,away_team,[<class 'str'>],100.0,0.0,0
3,team,[<class 'str'>],100.0,0.0,0
4,scorer,[<class 'str'>],100.0,0.0,0
5,minute,[<class 'float'>],100.0,0.0,0
6,own_goal,"[<class 'bool'>, <class 'float'>]",100.0,0.0,1
7,penalty,"[<class 'bool'>, <class 'float'>]",100.0,0.0,1


Se observa que hay dos tipos de datos para las columnas own_goal y penalty, se exploran los dos campos:

In [52]:
unique_own_goal = df["own_goal"].unique()
unique_penalty = df["penalty"].unique()

print("Valores únicos de la columna 'own_goal':", unique_own_goal)
print("Valores únicos de la columna 'penalty':", unique_penalty)

Valores únicos de la columna 'own_goal': [False True nan]
Valores únicos de la columna 'penalty': [False True nan]


In [54]:
# Eliminamos los nan
df.dropna(inplace=True)

In [57]:
print(df.shape)
ut.verificar_tipo_datos_y_nulos(df)  # Verificamos que se hayan eliminado los nulos

(44059, 8)


,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,date,[<class 'str'>],100.0,0.0,0
1,home_team,[<class 'str'>],100.0,0.0,0
2,away_team,[<class 'str'>],100.0,0.0,0
3,team,[<class 'str'>],100.0,0.0,0
4,scorer,[<class 'str'>],100.0,0.0,0
5,minute,[<class 'float'>],100.0,0.0,0
6,own_goal,[<class 'bool'>],100.0,0.0,0
7,penalty,[<class 'bool'>],100.0,0.0,0


In [67]:
# Contar filas duplicadas
filas_duplicadas = df.duplicated().sum()

# Mostrar el número de filas duplicadas
print(f"Número de filas duplicadas en el DataFrame: {filas_duplicadas}")

Número de filas duplicadas en el DataFrame: 82


## Variables categóricas y numéricas 

In [63]:
# Obtener las columnas categóricas
cols_cat = df.select_dtypes(include=["object"]).columns.tolist()

# Obtener las columnas numéricas
cols_num = df.select_dtypes(include=["float64", "int64"]).columns.tolist()

print("Columnas categóricas:", cols_cat)
print("Columnas numéricas:", cols_num)

Columnas categóricas: ['date', 'home_team', 'away_team', 'team', 'scorer', 'own_goal', 'penalty']
Columnas numéricas: ['minute']


In [65]:
# Conteo de subniveles en columnas categóricas
for col in cols_cat:
    print(f"Columna {col}: {df[col].nunique()} subniveles")

Columna date: 4599 subniveles
Columna home_team: 220 subniveles
Columna away_team: 220 subniveles
Columna team: 220 subniveles
Columna scorer: 14249 subniveles
Columna own_goal: 2 subniveles
Columna penalty: 2 subniveles


In [61]:
scorer_counts = df["scorer"].value_counts()
print(scorer_counts.head(10))

scorer
Cristiano Ronaldo      111
Robert Lewandowski      62
Romelu Lukaku           61
Harry Kane              55
Lionel Messi            54
Edin Džeko              50
Ali Daei                49
Miroslav Klose          48
Aleksandar Mitrović     47
Carlos Ruiz             47
Name: count, dtype: int64
